In [ ]:
import pandas as pd
import numpy as np
import math
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import networkx.algorithms.bipartite as bp

In [ ]:
data_role = "https://raw.githubusercontent.com/theoracley/Data620/master/Project2/monero_crime_person_role.csv"
data =pd.read_csv(data_role, header=None)
uniqueRoles = set(list(data[0]))
uniqueRoles

In [ ]:
data_crime ="https://raw.githubusercontent.com/theoracley/Data620/master/Project2/out.moreno_crime_crime"
#data_role = "monero_crime_person_role.csv"
crime = pd.read_csv(data_crime, delim_whitespace=True, header=None, skiprows=[0,1], names=['Person', 'Crime'])
crime['Rel_Role'] = pd.read_csv(data_role, header=None)
crime.head()

In [ ]:

crime.count()

In [ ]:
len(set(crime.Person))

In [ ]:
len(set(crime.Crime))

In [ ]:
# checking relationship Roles 
Relations = crime.groupby('Rel_Role')
Relations.count()

#Relations.count().iloc[:,0]

#Crimes By Person
crimes_per_person = crime.groupby('Person')
crimes_per_person.count()[:, 1]

## Crime Distribution (by Person)

##### Accordong to the graph below, most Persons were involved in a single crime.


In [ ]:

# histogram [count of Crimes]
crime_count = pd.DataFrame({'Crime_Count' : crime.groupby(['Person']).size()}).reset_index()
crime_count.iloc[:,1].plot.hist(alpha=0.5, bins=30)
plt.xlabel('Crimes', fontsize=20)
plt.ylabel('Persons Involved', fontsize=20)
plt.title('Histogram (Persons involved by Crime)')
plt.grid(True)
plt.rcParams['figure.figsize'] = (25, 10)
plt.show()

#### For every person, let's see how many crimes he/she was involved in, and how many Roles he/she had. We'll add the count information to every row, for quick access.

In [ ]:
extended_crime = crime
#how many Crimes
extended_crime['num_crimes'] = extended_crime.groupby(['Person'])['Crime'].transform('count')

#how many Roles
extended_crime['num_rel_role'] = extended_crime.groupby(['Person', 'Rel_Role'])['Crime'].transform('count')

extended_crime.head()

#### let's make the records meaningful by adding corresponding person name and sex, since do have data for that. 


In [ ]:
#get name data
data_name = 'https://raw.githubusercontent.com/theoracley/Data620/master/Project2/ent.moreno_crime_crime.person.name'
names = pd.read_csv(data_name, sep='\t', header=None)

#get sex data
data_sex = 'https://raw.githubusercontent.com/theoracley/Data620/master/Project2/ent.moreno_crime_crime.person.sex'
names['Sex'] = pd.read_csv(data_sex, header=None)

names.ix[names.Sex == 1, 'Sex'] = 'M'
names.ix[names.Sex == 0, 'Sex'] = 'F'


In [ ]:
extended_crime["Sex"] = ""
# replacing left nodes (people involved in a crime) with names and adding sex
# Right nodes stay as numbers to identify as crimes, not people
for i in range(0,len(names)):
    extended_crime.ix[extended_crime.Person == i+1, 'Sex'] = names.iloc[i]["Sex"]
    extended_crime.ix[extended_crime.Person == i+1, 'Person'] = names.iloc[i][0]
    
extended_crime.head()

## Relationship Roles

#### There are lot of overlaps. Many suspects are also victims, and many Victimes are also witnesses. The other way is also true.
#### For example Katz who is a criminal Pro, has been involved in 18 crimes, 2 victims, once as a witness and 15 times as a suspect.

In [ ]:
## Grouping suspect, victim and witness nodes in groups, and compute the overlap
suspects = set(extended_crime[extended_crime['Rel_Role'] == "Suspect"].iloc[:, 0].unique())
victims = set(extended_crime[extended_crime['Rel_Role'] == 'Victim'].iloc[:,0].unique())
witness = set(extended_crime[extended_crime['Rel_Role'] == "Witness"].iloc[:, 0].unique())

sus_len = len(suspects)
vict_len = len(victims)
wit_len = len(witness)

sus_vict = len(suspects.intersection(victims))
sus_wit = len(suspects.intersection(witness))
vict_wit = len(victims.intersection(witness))

print('') 
print('Suspect | Victim | Witness Overlap')
print ('----------------------------------')
print ('')
print ('Suspects:               ', sus_len)
print ('Victims:                ', vict_len)
print ('Witnesses:              ', wit_len)
print ('')
print ('Suspect and Victim:      ', sus_vict)
print ('Suspect and Witness:     ', sus_wit)
print ('Victim and Witness:      ', vict_wit)
print ('Victim and Witness:      ', vict_wit)

## Bipartite Graph
#### Monu and I had problems loading the data automatically from pandas frames through networkx API. 
#### After struggle, we decided to do it manually by using add nodes and add edges methods.

In [ ]:
## reloading data in bipartite fashion
G2 = nx.Graph()
# adding Person nodes
for i in range(len(extended_crime)): 
    G2.add_node(extended_crime.iloc[i][0],
                Relation=extended_crime.iloc[i][2],
                rel_count=extended_crime.iloc[i][3],
                weight=extended_crime.iloc[i][4],
                Sex=extended_crime.iloc[i][5],
                bipartite=1)


In [ ]:
# add crime nodes
for i in range(len(extended_crime)): 
    G2.add_node(extended_crime.iloc[i][1],
                bipartite=0)

In [ ]:
# add edges
# There's an edge between two stations if there was a trip between them.
# Edges are weighted by the number of trips
for i in range(len(extended_crime)):
    G2.add_edge(extended_crime.iloc[i][0], extended_crime.iloc[i][1], weight=1)

In [ ]:
# is this graph connected. 
nx.is_connected(G2)

In [ ]:
#Is it bipartite
nx.is_bipartite(G2)

In [ ]:
# Generaing a list of top_nodes(people) and bottom_nodes(crime)
top_nodes = set(n for n,d in G2.nodes(data=True) if d['bipartite']==1)
bottom_nodes = set(G2) - top_nodes

## Examine this!!!

In [ ]:
#get the info about our graph
print(nx.info(G2))

In [ ]:
# Printing the first 15 individuals in the dataset
print ("First 15 Persons:")

for i in range(10):
    print (' %s    ' % (list(top_nodes)[i]))

## Graph this bipartite


In [ ]:
# ## plot the overall network
pos=nx.random_layout(G2) # positions for all nodes

# ## elarge=[(u,v) for (u,v,d) in G.edges(data=True) if d['Count'] > 10]
# ## esmall=[(u,v) for (u,v,d) in G.edges(data=True) if d['Count'] <= 10]

colors=[]
for n in dict(G2.degree()).keys():    
    if n in dict(G2.degree(top_nodes)).keys():
        colors.append('r')
    else:
        colors.append('b')

# # nodes
d = G2.degree()
nx.draw_networkx_nodes(G2,pos, node_color=colors, node_size=[v*400 for v in dict(d).values()], alpha=0.75)

# edges
nx.draw_networkx_edges(G2,pos, width=4,alpha=0.5,edge_color='g',style='solid')
#nx.draw_networkx_edges(G2,pos,edgelist=esmall, width=2,alpha=0.2,edge_color='b',style='dashed')

# labels
nx.draw_networkx_labels(G2, pos, font_size=8, font_family='sans-serif')

plt.axis('off')
plt.show()

## Person - Bipartite Projection

#### 1. Island Moethod and analysis 

We will use the island method after the creation of the weighted bipartite graph. Weights are the number of shared neighbors for an edge. First we isolate the biggest component of the bipartite graph, then we apply the island method and print the threshold level, the size of the graph, and the number of connected components.

In [ ]:
## SNAFS: get a sorted degree list to obtain topmost nodes
def sorted_map(map):
    ms = sorted(map.iteritems(), reversed = True) #key=lambda (k,v) : (-v, k))
    return ms

## Implementing the Island Method to look for important subgroups. ##
def trim_edges(g, weight = 1):
    # Implements a trimming of the graph edges for the Island Analysis
    g2 = nx.Graph()
    for f, to, edata in g.edges(data=True):
        if edata['weight'] > weight:
            g2.add_edge(f,to)
    return g2

def island_method(g, iterations = 5):
    # Iterates through the graph removing edges and returning the number of edges removed and
    #  the trimmed graph
    weights = [edata['weight'] for f,to,edata in g.edges(data=True)]
    mn=int(min(weights))
    mx=int(max(weights))
    step = int((mx-mn)/iterations)
    return [[threshold, trim_edges(g, threshold)] for threshold in range(mn,mx)]

#### 2. Island method results

A threshold value of 2 results in a network of 28 Persons and 12 connected components. Of particular interest are suspect nodes connected to Luella Katz, the most-connected suspect in the graph with a degree centrality of 51. Katz is also connected to two other high-degree suspects, Michael Thomas Smith (33 degrees) and Catherine Steiner (29). Both are in the top 10 list of suspects by degree centrality.

In [ ]:
PC = nx.bipartite.weighted_projected_graph(G2, top_nodes)
cc = list(nx.connected_component_subgraphs(PC))[0]
print(cc)

In [ ]:
## Running the Island Analysis on the graph. Will throw an error if the island_method is called with too many steps
islands = island_method(cc,3)
print ('')
print ('Threshold |  Size (Persons)  |  Connected Components')
print ('******************************************************')
for island in islands:
    print('   %d               %2d                   %2d' 
          %(island[0], len(island[1]), len(list(nx.connected_component_subgraphs(island[1]))))) 

#### 3. Graph this at a treshhold = 2.

In [ ]:
## Plot created by the island analysis
pc = trim_edges(cc, 2)

nodecolors=[]
for n in dict(pc.degree(top_nodes)).keys():
    if n in suspects:
        nodecolors.append('g')
    elif n in victims:
        nodecolors.append('b')
    else:
        nodecolors.append('y')

dpc = pc.degree()
nx.draw_random(pc, node_size=[v*1200 for v in dict(dpc).values()], with_labels=True, node_color=nodecolors, 
               alpha=.25, font_size=20)

## 4. What about the person Katz. Let's give him a visit and check his ego network.

#### Katz has 51 connections to other people. Those persons are themselves involved as suspects in 40 crimes and victimes in 22 crimes.

In [ ]:
luella_katz = nx.ego_graph(PC, 'KatzLuella', radius=1, center=True, undirected=True, distance='weight')

nodecolors=[]
for n in dict(pc.degree(top_nodes)).keys():
    if n in suspects:
        nodecolors.append('g')
    elif n in victims:
        nodecolors.append('b')
    else:
        nodecolors.append('y')

dkatz = luella_katz.degree()
nx.draw_random(pc, node_size=[v*1200 for v in dict(dkatz).values()], with_labels=True, node_color=nodecolors, 
               alpha=.25, font_size=20)


In [ ]:
luella_katz = extended_crime[extended_crime['Person']=='KatzLuella']
katz_connections = pd.DataFrame(cc.edges('KatzLuella'))
katz_con = set(katz_connections.iloc[:,1])
a = len(katz_con.intersection(victims))
b = len(katz_con.intersection(suspects))

print ('Luella Katz:')
print ('***************************')
print ('Victim connections:  ', a)
print ('Suspect connections: ', b)



## Crimes - Bipartite Projection

#### Let's now take the set of Crime nodes and the relation between them based on the same person.

In [ ]:
CC = nx.bipartite.weighted_projected_graph(G2, bottom_nodes)

#### 1. Crimes - island Analysis

The island analysis identifies a six-node network of crimes. Examination of the crimes shows they all involved Luella Katz and close associates who were suspects. Katz was a witness in crime 110 along with 14 other individuals.

In [ ]:
## Run the Island Analysis on the graph
islands = island_method(CC, iterations = 2)

print ('Depth  Crime  Networks')
print ('************************')
for island in islands:
    print(' %d      %3d      %2d' 
          %(island[0], len(island[1]), len(list(nx.connected_component_subgraphs(island[1])))))
    

#### 2. Graph This!!

In [ ]:
# 2 graphs created by the island analysis:
wc = trim_edges(CC, 2)
dwc = wc.degree()
nx.draw(wc, node_size=[v*1400 for v in dict(dwc).values()], with_labels=True, node_color='b', alpha=.25, font_weight='bold', font_size=20)


## Centrality Measures

In the table below we take the bipartite weighted graph and examine centrality measures. Per our visual analysis, Luella Katz's heavy involvement is confirmed; she ranks among the top 10 three centrality categories. Katz, Cathering Steiner and Michael Smith all are among the highest in eigenvector centrality – in other words, they are connected to other highly connected suspects and victims. Jenny Willis ranks high in betweenness and degree centrality. A plot of her ego network is below. Inspection shows that Willis and Katz were not involved in any of the same crimes.

Crime 110 is of note for high betweeeness centrality: The crime involved 15 witnesses, 2 suspects and one victim.


In [ ]:
## utility function to get topmost nodes for each centrality function
def get_top_keys(c, top):
    #items = dictionary.items()
    items= sorted(dictionary.items(), reverse=True, key=lambda x: x[1])
    return map(lambda x: x, items[:top])

# Betweenness centrality
bet_cen = nx.betweenness_centrality(G2)

# Degree centrality
deg_cen = nx.degree_centrality(G2)

# Closeness centrality
clo_cen = nx.closeness_centrality(G2)

# Eigenvector centrality
eig_cen = nx.eigenvector_centrality_numpy(G2)

## get the top nodes
top_bet_cen = get_top_keys(bet_cen,10)
top_clus_cen = get_top_keys(clo_cen,10)
top_eig_cen = get_top_keys(eig_cen,10)
top_deg_cen = get_top_keys(deg_cen,10)

## smush them together to make a comparison chart
smush = np.hstack((top_bet_cen, top_clus_cen, top_eig_cen, top_deg_cen))

cent_measures=pd.DataFrame(smush, columns=['Node', 'Betweenness', 'Node', 'Closeness', 'Node', 
                                           'Eigenvector', 'Node', 'Degree'])

In [ ]:
cent_measures